In [1]:
from urllib.request import urlretrieve

In [2]:
import os

In [3]:
os.getcwd()

'C:\\Users\\User'

In [6]:
urlretrieve("https://github.com/berislav-vidakovic/Portfolio/blob/main/PythonETL/", "exchange_rate.csv")

('exchange_rate.csv', <http.client.HTTPMessage at 0x1cbbb787fd0>)

In [14]:
import fnmatch

In [17]:
fnmatch.filter(os.listdir('.'), '*.csv')

['exchange_rate.csv', 'output.csv']

In [32]:
!python --version

Python 3.11.4


In [34]:
!pip install requests

In [35]:
!pip install bs4

In [36]:
!pip install pandas

In [41]:
!pip install numpy

In [42]:
!pip install datetime

  Obtaining dependency information for datetime from https://files.pythonhosted.org/packages/f3/78/8e382b8cb4346119e2e04270b6eb4a01c5ee70b47a8a0244ecdb157204f7/DateTime-5.5-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/52.6 kB ? eta -:--:--
   ---------------------------------------- 52.6/52.6 kB 1.4 MB/s eta 0:00:00


In [43]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup
import sqlite3
from datetime import datetime 